In [1]:
import pymysql
import re
import json
import pandas as pd

import os
import numpy as np
import warnings
import sys

In [2]:
def pre_process(text):
    """
    Preprocesses Tweet text for sentiment analysis.
    :param text:
    :return: cleaned text
    """
    text = str(text)
    # Remove links
    text = re.sub('http:\S+|https:\S+', '', text)
    text = re.sub(r"http\S+", "", text)

    # Convert HTML references
    text = re.sub('&amp', 'and', text)
    text = re.sub('&lt', '<', text)
    text = re.sub('&gt', '>', text)

    # Remove new line characters
    text = re.sub('\\n|[\r\n]+', ' ', text)

    # Remove multiple space characters
    text = re.sub('\s+', ' ', text)

    # Convert emojis to text
    # text = emoji.demojize(text, delimiters=("", ""))

    return str(text)


In [3]:
import json
import pandas as pd
import os
import numpy as np
import warnings
import sys

# removes SettingWithCopyWarning
pd.options.mode.chained_assignment = None

# removes Future Warning
warnings.simplefilter(action='ignore', category=FutureWarning)


# Json file cleaning function
def json_cleaning(filename):
    """
    Cleans JSON files to make them suitable for MySQL.
    :param filename:
    :return: cleaned dataframe
    """
    df = pd.read_json(open(filename, "r", encoding="utf8"), lines=True)
    df = df[
        ['created_at', 'id_str', 'text', 'in_reply_to_status_id_str', 'in_reply_to_user_id_str', 'user', 'coordinates',
         'quote_count', 'reply_count', 'retweet_count', 'entities', 'favorite_count', 'retweeted', 'filter_level',
         'lang', 'extended_tweet']]
    df.rename(columns={'created_at': 'tweet_created_at'}, inplace=True)

    # making extra datetime columns
    df['year_tweet'] = pd.DatetimeIndex(df['tweet_created_at']).year
    df['month_tweet'] = pd.DatetimeIndex(df['tweet_created_at']).month
    df['day_tweet'] = pd.DatetimeIndex(df['tweet_created_at']).day
    df['time_tweet'] = pd.DatetimeIndex(df['tweet_created_at']).time

    # add User Object to main dataframe
    df_user = df['user'].apply(pd.Series)
    df_user = df_user[['id_str', 'verified', 'followers_count']]
    df_user.columns = 'user_' + df_user.columns

    df = pd.concat([df_user, df.drop('user', axis=1)], axis=1)

    # add entities as columns to main dataframe
    df_entity = df['entities'].apply(pd.Series)
    df_entity = df_entity[['user_mentions', 'hashtags']]
    df_entity.columns = 'entity_' + df_entity.columns

    df = pd.concat([df_entity, df.drop('entities', axis=1)], axis=1)

    # add extended tweet to main dataframe
    df_extended_tweet = df['extended_tweet'].apply(pd.Series)
    df_extended_tweet = df_extended_tweet[['full_text']]
    df_extended_tweet.columns = 'extended_' + df_extended_tweet.columns

    df = pd.concat([df_extended_tweet, df.drop('extended_tweet', axis=1)], axis=1)

    # fixing User Mentions in entities
    isna = df['entity_user_mentions'].isna()
    df.loc[isna, 'entity_user_mentions'] = pd.Series([[]] * isna.sum()).values
    for index, info in df[['entity_user_mentions']].iterrows():
        str_list = []
        for dictionary in info[0]:
            name = dictionary['screen_name']
            mention_id = dictionary['id_str']
            value = (str(name), str(mention_id))
            str_list.append(value)
        f_value = ",".join(map(str, str_list))
        df['entity_user_mentions'][index] = f_value

    # fixing hashtags in entities
    isna2 = df['entity_hashtags'].isna()
    df.loc[isna2, 'entity_hashtags'] = pd.Series([[]] * isna2.sum()).values
    for index, info in df[['entity_hashtags']].iterrows():
        str_list = []
        for dictionary in info[0]:
            text = dictionary['text']
            str_list.append(text)
        f_value = ",".join(map(str, str_list))
        df['entity_hashtags'][index] = f_value

    # light cleaning on the tweets' text
    for index, info in df[['text']].iterrows():
        pro_text = pre_process(info[0])
        df['text'][index] = pro_text

    # light cleaning on the extended text
    for index, info in df[['extended_full_text']].iterrows():
        pro_text = pre_process(str(info[0]))
        df['extended_full_text'][index] = pro_text

    # removes wrong rows
    wrong_rows = df[df['user_id_str'].isna()]
    df = df.drop(index=list(wrong_rows.index))

    # cleaning coordinates columns
    df['coordinates'].fillna('', inplace=True)
    for index, info in df[['coordinates']].iterrows():
        row = dict(info[0])
        coordinates = ''
        for key, value in row.items():
            if key == 'coordinates':
                coordinates = str(value)
        df.loc[index, 'coordinates'] = coordinates

    # making the texts one column
    df['full_text'] = np.nan
    for index, info in df[['extended_full_text']].iterrows():
        if info[0] != 'nan':
            df['full_text'][index] = info[0]
        else:
            df['full_text'][index] = df['text'][index]
    df = df.drop(columns=['extended_full_text', 'text'])

    # correcting column types
    df = df.where(df.notnull(), None)
    df['user_followers_count'] = df['user_followers_count'].astype(int)
    df['id_str'] = df['id_str'].astype(str)
    df['user_id_str'] = df['user_id_str'].astype(str)
    df['quote_count'] = df['quote_count'].astype(int)
    df['reply_count'] = df['reply_count'].astype(int)
    df['retweet_count'] = df['retweet_count'].astype(int)
    df['favorite_count'] = df['favorite_count'].astype(int)
    df['retweeted'] = df['retweeted'].astype(int)
    df['coordinates'] = df['coordinates'].astype(str)
    df['year_tweet'] = df['year_tweet'].astype(int)
    df['month_tweet'] = df['month_tweet'].astype(int)
    df['day_tweet'] = df['day_tweet'].astype(int)
    df['in_reply_to_user_id_str'] = df['in_reply_to_user_id_str'].astype(str)
    df['in_reply_to_status_id_str'] = df['in_reply_to_status_id_str'].astype(str)

    # drop duplicate rows
    df.drop_duplicates()

    return df


In [19]:
final = df[0:0]

In [20]:
wrong_files = []

In [5]:
# df = json_cleaning('data/1/1.1\\airlines-1585470568347.json')

CHANGE the path to the folder where all the data is

In [50]:
path = 'data/3' #add the path of the folder on your own computer
files_list = []
for filename in os.listdir(path):
    f = os.path.join(path, filename)
    if os.path.isfile(f):
        files_list.append(f"{f}")
print(files_list)


for file in files_list:
    try:
        result = json_cleaning(file)
        df_start = result[(result['in_reply_to_user_id_str']=='nan')&(result['in_reply_to_status_id_str']=='nan')]
        df_valid = result[(result['in_reply_to_user_id_str']!='nan')&(result['in_reply_to_status_id_str']!='nan')]
        df_all = pd.concat([df_start, df_valid])
        final = final.append(df_all, ignore_index=True)
    except:
        wrong_files.append(file)


['data/3\\airlines-1560978307300.json', 'data/3\\airlines-1561022289578.json', 'data/3\\airlines-1561049226682.json', 'data/3\\airlines-1561082554486.json', 'data/3\\airlines-1561113923197.json', 'data/3\\airlines-1561144186198.json', 'data/3\\airlines-1561207094360.json', 'data/3\\airlines-1561272936313.json', 'data/3\\airlines-1561288155121.json', 'data/3\\airlines-1561303000123.json', 'data/3\\airlines-1561324694075.json', 'data/3\\airlines-1561359351927.json', 'data/3\\airlines-1561397411189.json', 'data/3\\airlines-1561459414825.json', 'data/3\\airlines-1561509319618.json', 'data/3\\airlines-1561544763662.json', 'data/3\\airlines-1561592689779.json', 'data/3\\airlines-1561669318337.json', 'data/3\\airlines-1561753469078.json', 'data/3\\airlines-1561821435981.json', 'data/3\\airlines-1561891783071.json', 'data/3\\airlines-1561961961307.json', 'data/3\\airlines-1562009333840.json', 'data/3\\airlines-1562032544994.json', 'data/3\\airlines-1562054794843.json', 'data/3\\airlines-156208

C:\Users\hetvi\AppData\Local\Temp/ipykernel_13364/768326419.py:58: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  df.loc[isna, 'entity_user_mentions'] = pd.Series([[]] * isna.sum()).values
C:\Users\hetvi\AppData\Local\Temp/ipykernel_13364/768326419.py:71: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  df.loc[isna2, 'entity_hashtags'] = pd.Series([[]] * isna2.sum()).values


In [52]:
print(wrong_files)

['data/1/1.3\\airlines-1583171908044.json', 'data/1/1.3\\airlines-1583253051533.json', 'data/2\\airlines-1558611772040.json', 'data/2\\airlines-1558623303180.json', 'data/2\\airlines-1559860041436.json', 'data/2\\airlines-1560138591670.json', 'data/3\\airlines-1565894560588.json', 'data/3\\airlines-1569957146471.json', 'data/3\\airlines-1570104381202.json', 'data/3\\airlines-1573229502947.json', 'data/3\\airlines-1575313134067.json']


In [54]:
final.head()

,entity_user_mentions,entity_hashtags,user_id_str,user_verified,user_followers_count,tweet_created_at,id_str,in_reply_to_status_id_str,in_reply_to_user_id_str,coordinates,...,retweet_count,favorite_count,retweeted,filter_level,lang,year_tweet,month_tweet,day_tweet,time_tweet,full_text
0,"('flightradar24', '134196350')",,2817158784,False,4278,2020-03-29 08:29:22+00:00,1.2441798340388209e+18,nan,nan,,...,0,0,0,low,en,2020,3,29,08:29:22,RT @flightradar24: KLM🇳🇱 and Qantas🇦🇺 have eac...
1,"('breakingavnews', '3781510577')",,281123499,False,96,2020-03-29 08:29:27+00:00,1.2441798554004726e+18,nan,nan,,...,0,0,0,low,en,2020,3,29,08:29:27,RT @breakingavnews: Qantas' last ever 747 flig...
2,"('deemadigan', '35619835')",,926253818656018433,False,90,2020-03-29 08:29:29+00:00,1.2441798614400942e+18,nan,nan,,...,0,0,0,low,en,2020,3,29,08:29:29,RT @deemadigan: An ex-Virgin staff received th...
3,"('alfredblokhuize', '89764219'),('bergfrans', ...",,965640316907937792,False,646,2020-03-29 08:29:34+00:00,1.244179883561046e+18,nan,nan,,...,0,0,0,low,nl,2020,3,29,08:29:34,RT @alfredblokhuize: @bergfrans @RichardAbbenh...
4,"('AlwhiteBlanc', '1137101751671250944')","PontAerien,AF6901,masques",2632360764,False,2881,2020-03-29 08:29:35+00:00,1.2441798863376384e+18,nan,nan,,...,0,0,0,low,fr,2020,3,29,08:29:35,RT @AlwhiteBlanc: [#PontAerien 🇨🇳✈️]👉 #AF6901 ...


In [55]:
final.to_csv('final_final.csv',index=False)